In [1]:
# I have a llama pretraind model in /data/llm/llama/Llama-3.2-1B-Instruct, Load it and use it to generate text
# I have a llama pretraind model in /data/llm/llama/Llama-3.2-1B-Instruct, Load it and use it to generate text
CHECKPOINT_DIR="/data/llm/llama/Llama-3.2-1B-Instruct/original"
LLAMA_MODEL_MODULES_PATH="/raid/students/gerry/repos/llama-models"

In [2]:
# Add LLAMA_MODEL_MODULES_PATH to sys.path
import sys
if LLAMA_MODEL_MODULES_PATH not in sys.path:
    sys.path.append(LLAMA_MODEL_MODULES_PATH)


In [3]:
from pathlib import Path
SCRIPTS_DIR = Path(LLAMA_MODEL_MODULES_PATH)/"models/scripts"
TOKENIZER_PATH = str(SCRIPTS_DIR.parent / "llama3/api/tokenizer.model")
TOKENIZER_PATH

'/raid/students/gerry/repos/llama-models/models/llama3/api/tokenizer.model'

In [4]:
import os
os.environ['RANK'] = str(0)
os.environ['WORLD_SIZE'] = str(1)
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12355'

In [5]:
from pathlib import Path
from typing import Optional

import fire

from models.llama3.api.datatypes import (
    CompletionMessage,
    StopReason,
    SystemMessage,
    UserMessage,
)

from models.llama3.reference_impl.generation import Llama

In [6]:
def setup_llama(
    ckpt_dir: str,
    temperature: float = 0.6,
    top_p: float = 0.9,
    max_seq_len: int = 512,
    max_batch_size: int = 4,
    max_gen_len: Optional[int] = None,
    model_parallel_size: Optional[int] = 1,
):
    """
    Examples to run with the models finetuned for chat. Prompts correspond of chat
    turns between the user and assistant with the final one always being the user.

    An optional system prompt at the beginning to control how the model should respond
    is also supported.

    `max_gen_len` is optional because finetuned models are able to stop generations naturally.
    """
    tokenizer_path = str(TOKENIZER_PATH)
    generator = Llama.build(
        ckpt_dir=ckpt_dir,
        tokenizer_path=tokenizer_path,
        max_seq_len=max_seq_len,
        max_batch_size=max_batch_size,
        model_parallel_size=model_parallel_size,
    )
    return generator

In [7]:
# Create a gradio interface to interact with the model
import gradio as gr

def chat_completion(text):
    llama = setup_llama(CHECKPOINT_DIR)

    result = llama.chat_completion(
        [UserMessage(content=text)],
        temperature=0.6,
        top_p=0.9,
        max_gen_len=512,
    )
    #print(result.generation)
    out_message = result.generation
    return f"> {out_message.role.capitalize()}: {out_message.content}"

def main():
    gr.Interface(fn=chat_completion, 
                 inputs="text",
                 outputs="text",
                 title="Llama 3.2 Chat (AI 231) 🦙",
                 description="Chat with the Llama 3.2 model sample",
                 allow_flagging="never").launch(inline=True, server_port=7864)
main()

/raid/students/gerry/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1


/raid/students/gerry/venv/lib/python3.8/site-packages/torch/__init__.py:955: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:432.)
  _C._set_default_tensor_type(t)


Loaded in 1.62 seconds
